In [ ]:
%pip install openai
%pip install azure-cognitiveservices-speech

In [ ]:
from openai import AzureOpenAI, BadRequestError
import os
import json

clinicalScenario = "abdominal pain"
patientDem = "50 year female"
patientCharacteristics = "expresses thoughts in non-linear manner and at a 6th grade level of understanding."

# Initialize the Azure OpenAI client for GPT-4
os.environ["AZURE_OPENAI_API_KEY"] = "[YOUR_AZURE_OPENAI_API_KEY]"
client = AzureOpenAI(
    api_version="2024-02-15-preview",
    azure_endpoint="[YOUR_AZURE ENDPOINT]",
)

# Function to prepare and send the prompt to GPT-4
def createNote(patientDemographics, clinicalScenario, progressNote, dictationMode="traditional", additionalContext=""):
    prompt = ""
    if dictationMode == "traditional":
        prompt = [
        {
            "role": "system",
            "content": """You are a clinical informatics professional responsible for creating example, realistic physician progress notes to be used for EHR platform testing.  You will be given a description of a patient along with a clinical symptom or diagnosis and then you will create a realistic encounter note that includes ONLY the following sections:  Chief Complaint or Reason for Visit, History of Present Illness, Past Medical History, Past Surgical History, Family History, Current Medications, Allergies, Social History, Review of Systems, Physical Exam, Test Results, Assessment and Plan

            - Depending on the scenario, certain sections will not be relevant and should not be included.
            - The naming of these section can vary to emulate the variability in physician dictated notes.
            - Make sure that any patient orders, such as prescriptions or diagnostic testing, are described in detail in the transcript so that they can be transcribed into the EHR by the nurse
            - The note should be written in the first person as if the physician is dictating the note.  This means that the format should be narrative and not bullet points.

            The output should be in this JSON format:
            {
            "result": "Chief Complaint: Abdominal pain\n\nHistory of Present Illness: The patient..."
            }
            """
        },
        {
            "role": "user",
            "content": f"""Create a progress note for a {patientDemographics} with {clinicalScenario}."""
        }
        ]
    else:
        if(additionalContext != ""):
            additionalContext = f"Create a realistic conversation transcript that accounts for the fact that the patient  {additionalContext}.\n"

        prompt = [
        {
            "role": "system",
            "content": """You will read a physician progress note and then create a realistic transcript of the encounter conversation between the patient and provider which lead to the creation of the note.  You will imagine that an audio recording was made of the conversation. The patient demographic information will be included in the note. Use the demographic information to enhance the realism of the conversation between the patient and provider. 
            
            The output should be in this JSON format:
            {
                "statements": [
                    {
                        "role": "provider",
                        "content": "Hello, how can I help you today?"
                    },
                    {
                        "role": "patient",
                        "content": "I have been having abdominal pain for a few days now."
                    },
                    {
                        "role": "provider",
                        "content": "I see. Can you tell me more about the pain? Is it sharp or dull?"
                    },
                    {
                        "role": "patient",
                        "content": "It's a sharp pain that comes and goes."
                    },
                    {
                        "role": "provider",
                        "content": "Do you have any other symptoms?"
                    },...
                ]
            }
            """
        },
        {
            "role": "user",
            "content": f"""Here is the physician progress note:\n {progressNote}\n\n{additionalContext}"""
        }
        ]
    

    try:
        response = client.chat.completions.create(
            model="[YOUR_MODEL]",
            messages=prompt,
            temperature=1,
            max_tokens=4095,
            top_p=1,
            response_format={"type": "json_object"},
            frequency_penalty=0,
            presence_penalty=0
        )

        if response is None or response.choices[0].message.content is None:
            print("Received empty response from API.")
            return None

        content = response.choices[0].message.content.strip()
        parsed_content = json.loads(content)
        return parsed_content
    except BadRequestError as e:
        print(f"BadRequestError occurred: {str(e)}")
        return "ERROR"
    except Exception as e:
        print(f"An unexpected error occurred: {str(e)}")
        return "ERROR"


# Call the function to generate the progress note;
# the function will return the generated progress note
progressNote = createNote(patientDem, clinicalScenario, "", dictationMode="traditional")
print(progressNote)

conversation = createNote(patientDem, clinicalScenario, progressNote, dictationMode="ambient", additionalContext=patientCharacteristics)
print(conversation)


{'result': "Chief Complaint: Abdominal pain\n\nHistory of Present Illness: The patient is a 50-year-old female presenting with abdominal pain that started approximately 3 days ago. She describes the pain as a constant, dull ache located in the lower right quadrant of her abdomen. The pain intensity is rated 6/10 and has gradually worsened since its onset. She has experienced associated symptoms of mild nausea and decreased appetite but denies vomiting, diarrhea, or fever. The patient notes that the pain is somewhat alleviated by lying down and worsens with movement or palpation.\n\nPast Medical History: The patient has a history of hypertension and hyperlipidemia. Both conditions are well-controlled with medication.\n\nPast Surgical History: The patient underwent a laparoscopic cholecystectomy 5 years ago with no complications.\n\nFamily History: The patient's mother had a history of breast cancer, and her father had a history of type 2 diabetes. No known family history of gastrointest

# Simulate Provider Dictation

In [14]:
import os
import azure.cognitiveservices.speech as speechsdk

# Creates an instance of a speech config with specified subscription key and service region.
speech_key = "[YOUR_SPEECH_KEY]"
service_region = "eastus"

# This example requires environment variables named "SPEECH_KEY" and "SPEECH_REGION"
speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=service_region)
audio_config = speechsdk.audio.AudioOutputConfig(use_default_speaker=True)

# The neural multilingual voice can speak different languages based on the input text.
speech_config.speech_synthesis_voice_name='en-US-AvaMultilingualNeural'

speech_synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config, audio_config=audio_config)

# Get text from the console and synthesize to the default speaker.
text = progressNote['result']

speech_synthesis_result = speech_synthesizer.speak_text_async(text).get()

if speech_synthesis_result.reason == speechsdk.ResultReason.SynthesizingAudioCompleted:
    print("Speech synthesized for text [{}]".format(text))
elif speech_synthesis_result.reason == speechsdk.ResultReason.Canceled:
    cancellation_details = speech_synthesis_result.cancellation_details
    print("Speech synthesis canceled: {}".format(cancellation_details.reason))
    if cancellation_details.reason == speechsdk.CancellationReason.Error:
        if cancellation_details.error_details:
            print("Error details: {}".format(cancellation_details.error_details))
            print("Did you set the speech resource key and region values?")

# Simulate Patient Provider Conversation

In [ ]:
import os
import azure.cognitiveservices.speech as speechsdk

# Load Azure Cognitive Services Speech key and region from environment variables
# (for security purposes, don't hard-code these in production)
speech_key = "[YOUR_SPEECH_KEY]"
service_region = "eastus"

# Creates instances of speech configs with the specified subscription key and service region
speech_config_provider = speechsdk.SpeechConfig(subscription=speech_key, region=service_region)
speech_config_patient = speechsdk.SpeechConfig(subscription=speech_key, region=service_region)

# Define the voices for the physician and the patient
speech_config_provider.speech_synthesis_voice_name = 'en-US-AndrewMultilingualNeural'
speech_config_patient.speech_synthesis_voice_name = 'en-US-AvaNeural'

audio_config = speechsdk.audio.AudioOutputConfig(use_default_speaker=True)

# Create separate speech synthesizers for the provider and the patient
speech_synthesizer_provider = speechsdk.SpeechSynthesizer(speech_config=speech_config_provider, audio_config=audio_config)
speech_synthesizer_patient = speechsdk.SpeechSynthesizer(speech_config=speech_config_patient, audio_config=audio_config)


# Iterate over the conversation and synthesize each statement
for statement in conversation['statements']:
    # Select the appropriate synthesizer based on the role
    if statement['role'] == 'provider':
        synthesizer = speech_synthesizer_provider
    elif statement['role'] == 'patient':
        synthesizer = speech_synthesizer_patient
    
    # Synthesize the text
    text = statement['content']
    speech_synthesis_result = synthesizer.speak_text_async(text).get()

    if speech_synthesis_result.reason == speechsdk.ResultReason.SynthesizingAudioCompleted:
        print("Speech synthesized for text [{}]".format(text))
    elif speech_synthesis_result.reason == speechsdk.ResultReason.Canceled:
        cancellation_details = speech_synthesis_result.cancellation_details
        print("Speech synthesis canceled: {}".format(cancellation_details.reason))
        if cancellation_details.reason == speechsdk.CancellationReason.Error:
            if cancellation_details.error_details:
                print("Error details: {}".format(cancellation_details.error_details))
                print("Did you set the speech resource key and region values correctly?")